In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.text("StartDate", "")

StartDate = dbutils.widgets.get("StartDate")
print(StartDate)


20250831


In [0]:
import datetime
date = datetime.datetime.strptime(str(StartDate), "%Y%m%d")

year_val  = str(date.year)
month_val = str(f"{date.month:02}")
day_val   = str(f"{date.day:02}")

print(f"Processing Date  Year: {year_val}, Month: {month_val}, Day: {day_val}")

Processing Date  Year: 2025, Month: 08, Day: 31


In [0]:
from pyspark.sql.functions import *

def get_latest_file(path):
    """
    Get the latest modified file inside a given volume folder.

    :param path: Folder path (e.g., '/Volumes/ecommerce_catalog/source/src_data/')
    """
    files = dbutils.fs.ls(path)
    if not files:
        return None
    
    # Find the file with the largest modificationTime
    latest_file = sorted(files, key=lambda f: f.modificationTime, reverse=True)[0]
    return latest_file.path


In [0]:

from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("EcommerceETL")\
    .getOrCreate()

# Path of volume where raw CSV is uploaded
source_path = "/Volumes/ecommerce_catalog/source/src_data"

latest_file_path = get_latest_file(source_path)
print("Latest file path:", latest_file_path)


# Read CSV into Spark DataFrame
df_raw = (spark.read
          .format("csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .load(latest_file_path))


Latest file path: dbfs:/Volumes/ecommerce_catalog/source/src_data/ecommerce_sales_incremental_600.csv


In [0]:
bronze_base = "/Volumes/ecommerce_catalog/bronze/extracted_data/"
bronze_partition_path = f"{bronze_base}/year={year_val}/month={month_val}/day={day_val}/"
print(f"Bronze Path: {bronze_partition_path}")

df_raw.write.format("csv")\
    .mode("append")\
    .save(bronze_partition_path)

Bronze Path: /Volumes/ecommerce_catalog/bronze/extracted_data//year=2025/month=08/day=31/
